### 0. Validate pretrained
Берём предобученную ResNet32 (``pretrained_models/resnet32-d509ac18.th``). Посмотрим на accuracy у исходной модели.

In [3]:
%run stage_0/validate_pretrained.py

Files already downloaded and verified
Test: [0/79]	Time 9.200 (9.200)	Loss 0.2115 (0.2115)	Prec@1 93.750 (93.750)
Test: [50/79]	Time 0.332 (0.538)	Loss 0.4470 (0.3828)	Prec@1 90.625 (92.371)
 * Prec@1 92.630
 * Timec@1 0.493


### 1. Get initial weights
Берём предобученную ResNet32 (``pretrained_models/resnet32-d509ac18.th``). Вытаскиваем веса, кладём тензоры в ``weights/weigths_base.mat``.

In [20]:
%run stage_1/get_initial_weights.py

Затем расскладываем веса в CP-decomposition, использую NLS (non-linear least squares), с помощью ``cpd_nls`` из [Tensorlab](https://www.tensorlab.net) в MATLAB (примерный скрипт в ``MATLAB/script_matlab_decompose.m``). В файле ``weights/weights_nls.mat``

### 2. Fine-tune initial decomposition
Заменяем набор слоёв в моделе на декомпозированные (все фильтры 3x3) (``weigths/weigths_nls.mat``) с помощью ``cpd_nls``. Затем делаем файнтюнинг весов (``epochs=50``). Лучшая модель сохранена в ``decomposed/best_initial_decompose.th``. Заменённые слои можно посмотреть в ``functional.py``, в примере всего 6.

In [ ]:
%run stage_2/fine_tune_initial_decomposition.py

![initial_decomposition](stage_2/initial_decomposition.png)

Можно запустить ``stage_2/check.py``, чтобы посмотреть на структуру модели и accuracy на тестовой выборке y ``decompose/best_initial_decompose.th`` (это быстро). Хуже на 1.1 от исходной модели.

In [11]:
%run stage_2/check.py

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Sequential(
          (0_decomposed): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1)

Files already downloaded and verified
Test: [0/79]	Time 7.342 (7.342)	Loss 0.3062 (0.3062)	Prec@1 92.969 (92.969)
Test: [50/79]	Time 0.303 (0.508)	Loss 0.5858 (0.4408)	Prec@1 90.625 (91.330)
 * Prec@1 91.520
 * Timec@1 0.430


### 3. Extend Decomposed Kernels
Берём отфайнтьюниную модель с предыдущего шага (``decomposed/best_initital_decompose.th``). Увеличиваем факторы (``1_decomposed``, ``2_decomposed``) до размеров 1x21 и 21x1. Добавляем сигмы ``resnet_with_sigmas.py`` и обучаем вместе с сигмами. Модель с лучшим приближением сохранена в ``decompose/best_extended_decompose.th``.

In [ ]:
%run stage_3/extend_decomposed_kernels.py

![best_extended_decomposition](stage_3/extended_kernels_decomposition.png)

![sigmas](stage_3/sigmas_values.png)

Можно запустить ``stage_3/check.py``, чтобы посмотреть на структуру модели, значения сигм, соответствующие им размеры ядер и и accuracy на тестовой выборке y ``decompose/best_extended_decompose.th`` (это быстро). 

In [17]:
%run stage_3/check.py

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Sequential(
          (0_decomposed): Conv2d(16, 8, kernel_size=(1, 1), stride=(1, 1)

best sigmas:  [-1.0235847234725952, -1.6244574785232544, -0.6324220895767212, -1.0306727886199951, -0.5999158024787903, -0.45050451159477234]
best kernels_sz:  [9, 7, 11, 9, 11, 11]
Files already downloaded and verified
Test: [0/79]	Time 7.086 (7.086)	Loss 0.2054 (0.2054)	Prec@1 91.406 (91.406)
Test: [50/79]	Time 0.432 (0.575)	Loss 0.4664 (0.3881)	Prec@1 85.938 (88.817)
 * Prec@1 89.050
 * Timec@1 0.520


### 4. Get decomposed weights
Берём предобученную полученную на предыдущем шаге модель, применяем к весам маску и кропаем их до полученных размеров. Затем кладём декомпозиции весов в ``weights/decomposed_weights.mat``.

In [21]:
import warnings
warnings.filterwarnings("ignore", category=np.VisibleDeprecationWarning) 

%run stage_4/get_decomposed_weights.py

kernels_sz:  [9, 7, 11, 9, 11, 11]
sigmas:  [-1.0235847234725952, -1.6244574785232544, -0.6324220895767212, -1.0306727886199951, -0.5999158024787903, -0.45050451159477234]


Затем обратно композируем тензоры с помощью ``cpdgen`` из [Tensorlab](https://www.tensorlab.net) в MATLAB (примерный скрипт в ``MATLAB/script_matlab_compose.m``). В файле ``weights/weights_composed.mat``.

### 5. Final fine-tune
Заменяем набор слоёв в моделе на нормальные ``conv2`` с новыми размерами фильтров и весами, скомпозированными на предыдущем шаге. Делаем финальный файнтьюнинг (``epochs=50``). Лучшая модель сохранена в ``decomposed/best_final.th``.

In [ ]:
%run stage_5/final_fine_tune.py

DataParallel(
  (module): ResNet(
    (conv1): Conv2d(3, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (layer1): Sequential(
      (0): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (shortcut): Sequential()
      )
      (1): BasicBlock(
        (conv1): Conv2d(16, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(16, 16, kernel_size=(9, 9), stride=(1, 1), padding=(4, 4), bias=False)
       

Test: [50/79]	Time 0.404 (0.577)	Loss 0.3990 (0.3361)	Prec@1 90.625 (90.119)
 * Prec@1 90.080
 * Timec@1 0.520
best precision:  90.08
Epoch: [1][0/391]	Time 12.782 (12.782)	Data 10.005 (10.005)	Loss 0.0851 (0.0851)	Prec@1 98.438 (98.438)
Epoch: [1][50/391]	Time 2.878 (2.925)	Data 0.001 (0.197)	Loss 0.2397 (0.1652)	Prec@1 90.625 (94.393)
Epoch: [1][100/391]	Time 2.787 (2.848)	Data 0.001 (0.100)	Loss 0.1388 (0.1692)	Prec@1 96.094 (94.144)
Epoch: [1][150/391]	Time 3.821 (2.807)	Data 0.000 (0.067)	Loss 0.1967 (0.1674)	Prec@1 92.188 (94.221)
Epoch: [1][200/391]	Time 2.650 (2.842)	Data 0.002 (0.050)	Loss 0.1502 (0.1692)	Prec@1 93.750 (94.213)
Epoch: [1][250/391]	Time 2.799 (2.807)	Data 0.000 (0.041)	Loss 0.1251 (0.1661)	Prec@1 94.531 (94.273)
Epoch: [1][300/391]	Time 2.893 (2.851)	Data 0.001 (0.034)	Loss 0.1617 (0.1642)	Prec@1 92.969 (94.321)


![final](stage_5/final.png)

Можно запустить ``stage_5/check.py``, чтобы посмотреть на структуру модели и accuracy на тестовой выборке y ``decompose/best_final.th`` (это быстро). 

In [ ]:
%run stage_5/check.py